# Intro to SQL
Learn SQL for working with databases, using Google BigQuery to scale to massive datasets.

## Dataset & Reference

### Data
- [Hacker News](https://news.ycombinator.com/)
- [Chicago Crime](https://www.kaggle.com/chicago/chicago-crime)
- [OpenAQ](https://openaq.org/#/?_k=l7rozu)


### APIs and reference
https://cloud.google.com/bigquery/docs/reference

## Getting Started With SQL and BigQuery
Learn the workflow for handling big datasets with BigQuery and SQL.

In [3]:
# To use BigQuery, we'll import the Python package below:
from google.cloud import bigquery

![Client, Dataset, Table](https://i.imgur.com/biYqbUB.png)
<center>Ref: https://www.kaggle.com/dansbecker/getting-started-with-sql-and-bigquery </center>

### Client
The first step in the workflow is to create a `Client` object. As you'll soon see, this `Client` object will play a central role in retrieving information from BigQuery datasets.

In [ ]:
# Create a "Client" object.
client = bigquery.Client()

### Dataset
In BigQuery, each dataset is contained in a corresponding project. In this case, our `hacker_news` dataset is contained in the `bigquery-public-data` project. To access the dataset,

- We begin by constructing a reference to the dataset with the `dataset()` method.
- Next, we use the `get_dataset()` method, along with the reference we just constructed, to fetch the dataset.

In [ ]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

### Table
Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.

We use the `list_tables()` method to list the tables in the dataset.

In [ ]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print number of tables in the dataset
print(len(tables))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

Similar to how we fetched a dataset, we can fetch a table. In the code cell below, we fetch the `full` table in the `hacker_news` dataset.

In [ ]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

Each `SchemaField` tells us about a specific column (which we also refer to as a **field**). In order, the information is:

- The **name** of the column
- The **field type** (or datatype) in the column
- The **mode** of the column (`'NULLABLE'` means that a column allows NULL values, and is the default)
- A **description** of the data in that column

### Row
We can use the `list_rows()` method to check just the first five lines of of the `full` table to make sure this is right. (Sometimes databases have outdated descriptions, so it's good to check.) This returns a BigQuery `RowIterator` object that can quickly be converted to a pandas DataFrame with the `to_dataframe()` method.

In [ ]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

The `list_rows()` method will also let us look at just the information in a specific column. If we want to see the first five entries in the `by` column, for example, we can do that!

In [ ]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

## Select, From & Where
The foundational compontents for all SQL queries.

### SELECT ... FROM
![SELECT ... FROM](https://i.imgur.com/c3GxYRt.png)
<center>Ref: https://www.kaggle.com/dansbecker/select-from-where </center>

Note that when writing an SQL query, the argument we pass to **FROM** is not in single or double quotation marks (' or "). It is in backticks (`).

### WHERE ...
![WHERE ..](https://i.imgur.com/HJOT8Kb.png)
<center>Ref: https://www.kaggle.com/dansbecker/select-from-where </center>

### Submitting the query to the dataset
- the first step is to create a `Client` object.
- set up the query with the `query()` method.
- Next, we run the query and convert the results to a pandas DataFrame.

In [ ]:
# Create a "Client" object
client = bigquery.Client()

# Set up the query
query_job = client.query(query)

# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()

### Working with big datasets
Estimate the size of any query before running it  
To see how much data a query will scan, we create a `QueryJobConfig` object and set the `dry_run` parameter to `True`.

In [ ]:
# Create a QueryJobConfig object to estimate size of query without running it.
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs.
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

In [ ]:
# Only run the query if it's less than 1 MB.
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# Set up the query (will only run if it's less than 1 MB).
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame.
safe_query_job.to_dataframe()

## Group by, Having & Count  
Get more interesting insights directly from your SQL queries.

### COUNT()
**COUNT()** is an example of an **aggregate function**, which takes many values and returns one. (Other examples of aggregate functions include **SUM()**, **AVG()**, **MIN()**, and **MAX()**.)

### GROUP BY
**GROUP BY** takes the name of one or more columns, and treats all rows with the same value in that column as a single group when you apply aggregate functions like **COUNT()**.  

Note that because it tells SQL how to apply aggregate functions (like **COUNT()**), it doesn't make sense to use **GROUP BY** without an aggregate function. Similarly, if you have any **GROUP BY** clause, then all variables must be passed to either a

- **GROUP BY** command, or
- an aggregation function.

### GROUP BY ... HAVING
**HAVING** is used in combination with **GROUP BY** to ignore groups that don't meet certain criteria.

## Order By
Order your results to focus on the most important data for your use case.

### ORDER BY
**ORDER BY** is usually the last clause in your query, and it sorts the results returned by the rest of your query.  
You can reverse the order using the **DESC** argument (short for 'descending'). 

### Dates
There are two ways that dates can be stored in BigQuery: as a **DATE** or as a **DATETIME**.

The **DATE** format has the year first, then the month, and then the day. It looks like this:
```
YYYY-[M]M-[D]D
```
- ```YYYY```: Four-digit year
- ```[M]M```: One or two digit month
- ```[D]D```: One or two digit day  

The **DATETIME** format is like the date format ... but with time added at the end.

### EXTRACT
Often you'll want to look at part of a date, like the year or the day. You can do this with **EXTRACT**.

![EXTRACT](https://i.imgur.com/PhoWBO0.png)
![EXTRACT](https://i.imgur.com/A5hqGxY.png)
<center>Ref: https://www.kaggle.com/dansbecker/order-by </center>

## As & With
Organize your query for better readability. This becomes especially important for complex queries.

### AS
use **AS** to rename the columns generated by your queries, which is also known as **aliasing**.  
 
![AS](https://i.imgur.com/teF84tU.png)
<center>Ref: https://www.kaggle.com/dansbecker/as-with </center>

### WITH ... AS
On its own, **AS** is a convenient way to clean up the data returned by your query. It's even more powerful when combined with **WITH** in what's called a "common table expression".  

A **common table expression** (or **CTE**) is a temporary table that you return within your query. CTEs are helpful for splitting your queries into readable chunks, and you can write queries against them.

![WITH ... AS](https://i.imgur.com/3xQZM4p.png)
<center>Ref: https://www.kaggle.com/dansbecker/as-with </center>  

Also, it's important to note that CTEs only exist inside the query where you create them, and you can't reference them in later queries. So, any query that uses a CTE is always broken into two parts: (1) first, we create the CTE, and then (2) we write a query that uses the CTE.

## Joining Data
Combine data sources. Critical for almost all real-world data problems.

### JOIN
Using **JOIN**, we can write a query to create a table with just two columns.  

![JOIN](https://i.imgur.com/fLlng42.png)
<center>Ref: https://www.kaggle.com/dansbecker/joining-data </center>

# Advanced SQL
Take your SQL skills to the next level.

## JOINs and UNIONs

## Analytic Functions

## Nested and Repeated Data

## Writing Efficient Queires